### The Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/stars.csv')

In [3]:
df.sample(5)

,description,language,repo,stargazers,user
18144,Parsel lets you extract data from XML/HTML doc...,Python,scrapy/parsel,469,AllanLRH
329,Task Queue based on Redis,Go,meitu/lmstfy,31,robsalasco
11654,🔥 Open source static status page. Uses hyperfa...,HTML,cstate/cstate,354,owen800q
6282,🛠 All-in-one web-based IDE specialized for mac...,Jupyter Notebook,ml-tooling/ml-workspace,563,pushpendrapratap
3705,JSON Schema Based Editor,JavaScript,jdorn/json-editor,5238,alephmelo


### The Cleanup

In [4]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [10]:
df.sample(5)

,description,language,repo,stargazers,user
16372,Never use print for debugging again,Python,cool-RR/PySnooper,12726,boada
2785,The pytest framework makes it easy to write sm...,Python,pytest-dev/pytest,4924,javad94
6768,Turn Python scripts into handouts with Markdow...,Python,danijar/handout,1772,DahlitzFlorian
18831,Python composable command line interface toolkit,Python,pallets/click,8260,rlgomes
14709,As easy as HTTPie /aitch-tee-tee-pie/ 🥧 Modern...,Python,jakubroztocil/httpie,43419,andreztz


In [8]:
# shove into something that looks like this?
pd.DataFrame([
    [0, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1]
])

,0,1,2,3
0,0,0,1,0
1,0,1,1,0
2,1,0,0,1
3,0,0,0,1


### The Preparation

In [11]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [12]:
df.sample(5)

,repo
user,
ayarotsky,"streamlit/streamlit,ageron/handson-ml2,vinta/a..."
victordomingos,"ytdl-org/youtube-dl,encode/httpx,MasoniteFrame..."
Curlybear,"vinta/awesome-python,CorentinJ/Real-Time-Voice..."
dsfb,"django/django,iodide-project/pyodide,streamlit..."
mjhea0,"sebst/pythonic-news,plainas/flask-swagger-type..."


In [16]:
df.loc['Curlybear'].values.tolist()

['vinta/awesome-python,CorentinJ/Real-Time-Voice-Cloning,danijar/handout,ageitgey/face_recognition,jofpin/trape,grapheneX/grapheneX,gto76/python-cheatsheet,trekhleb/homemade-machine-learning,manrajgrover/halo,chris104957/maildown,psf/black,stewartmcgown/uds,TheAlgorithms/Python,cool-RR/PySnooper,Bogdanp/molten,nicolargo/glances,mkaz/termgraph,donnemartin/system-design-primer,chubin/cheat.sh,psf/requests-html,Zulko/moviepy,jarun/Buku,pypa/pipenv,soimort/you-get,public-apis/public-apis,HelloZeroNet/ZeroNet']

### The Model

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            neighbors = self.nn.kneighbors(Xt, return_distance=False)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [35]:
model = NNRecommender()
model.fit(df['repo'])

In [36]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [37]:
model.predict([df['repo'][42]])

[['ageron/handson-ml2',
  'smacke/subsync',
  'weskerfoot/DeleteFB',
  'dae/anki',
  'shengqiangzhang/examples-of-web-crawlers',
  'deepfakes/faceswap',
  'sloria/TextBlob',
  'scikit-learn/scikit-learn',
  'sherlock-project/sherlock',
  'donnemartin/interactive-coding-challenges',
  'fastai/fastai',
  'Miserlou/Zappa']]

### Under the Hood

In [23]:
df.iloc[0].values.tolist()

['python/cpython,hugapi/hug,HypothesisWorks/hypothesis,timothycrosley/portray,timothycrosley/hypothesis-auto,tiangolo/fastapi,bocadilloproject/bocadillo,tartiflette/tartiflette,encode/httpx,pytest-dev/pytest,tonybaloney/wily,sdispater/poetry,mahmoud/glom,psf/black,chubin/wttr.in']

In [24]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
nn = NearestNeighbors(n_neighbors=10)

In [26]:
nn.fit(X)
ind = nn.kneighbors(return_distance=False)
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

In [29]:
df.iloc[106].values

array(['timothycrosley/hypothesis-auto,timothycrosley/portray,sdispater/poetry,wyounas/homer,tiangolo/fastapi,psf/black,marshmallow-code/marshmallow,falconry/falcon,cookiecutter/cookiecutter'],
      dtype=object)

#### Pickle 

In [30]:
import pickle

In [31]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x11889d0d0>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [38]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [33]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['ytdl-org/youtube-dl',
  'shengqiangzhang/examples-of-web-crawlers',
  'mherrmann/fbs',
  'vinta/awesome-python',
  'donnemartin/system-design-primer',
  'plasticityai/supersqlite',
  'TheAlgorithms/Python',
  'Avik-Jain/100-Days-Of-ML-Code',
  'fastai/fastai',
  'minimaxir/big-list-of-naughty-strings',
  'sloria/TextBlob',
  'PySimpleGUI/PySimpleGUI',
  'scikit-learn/scikit-learn']]